# PyTorch-Alexnet-ONNX-Caffe2

## Use PyTorch pretrained model

In [1]:
import torch
import torchvision.models as models
import os

# Use an existing model from Torchvision, note it 
# will download this if not already on your computer (might take time)
net = models.alexnet(pretrained=True)
net.eval()

# Create some sample input in the shape this model expects
dummy_input = torch.ones(10, 3, 224, 224)

# # It's optional to label the input and output layers
# input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
# output_names = [ "output1" ]

# # Use the exporter from torch to convert to onnx 
# # model (that has the weights and net arch)
# model_path = os.path.join("model", model_name)
# torch.onnx.export(model, dummy_input, model_path, verbose=True, input_names=input_names, output_names=output_names)


In [2]:
import os
if not os.path.isdir("model"):
    os.mkdir("model")

model_name = "pytorch_alexnet.onnx"
model_path = os.path.join("model", model_name)
torch.onnx.export(net, 
                  dummy_input, 
                  model_path, 
                  verbose=True,
                  export_params=True
                 )

graph(%0 : Float(10, 3, 224, 224)
      %1 : Float(64, 3, 11, 11)
      %2 : Float(64)
      %3 : Float(192, 64, 5, 5)
      %4 : Float(192)
      %5 : Float(384, 192, 3, 3)
      %6 : Float(384)
      %7 : Float(256, 384, 3, 3)
      %8 : Float(256)
      %9 : Float(256, 256, 3, 3)
      %10 : Float(256)
      %11 : Float(4096, 9216)
      %12 : Float(4096)
      %13 : Float(4096, 4096)
      %14 : Float(4096)
      %15 : Float(1000, 4096)
      %16 : Float(1000)) {
  %17 : Float(10, 64, 55, 55) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%0, %1, %2), scope: AlexNet/Sequential[features]/Conv2d[0]
  %18 : Float(10, 64, 55, 55) = onnx::Relu(%17), scope: AlexNet/Sequential[features]/ReLU[1]
  %19 : Float(10, 64, 27, 27) = onnx::MaxPool[kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[2, 2]](%18), scope: AlexNet/Sequential[features]/MaxPool2d[2]
  %20 : Float(10, 192, 27, 27) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5],

## Import the ONNX model to Caffe2

In [3]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

# Load the ONNX ModelProto object. model is a standard Python protobuf object
model = onnx.load("model/pytorch_alexnet.onnx")

# prepare the caffe2 backend for executing the model this converts the ONNX model into a
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be
# availiable soon.
c2_rep = onnx_caffe2_backend.prepare(model)

In [4]:
import numpy as np

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph of the model itself contains inputs for all weight parameters, after the input image.
# Since the weights are already embedded, we just need to pass the input image.
# Set the first input.
W = {model.graph.input[0].name: dummy_input.data.numpy()}

# Run the Caffe2 net:
c2_out = c2_rep.run(W)
torch_out = net(dummy_input).detach().numpy()

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(c2_out[0], 
                               torch_out, 
                               decimal=3
                              )

print("Exported model has been executed on Caffe2 backend, and the result looks good!")

Exported model has been executed on Caffe2 backend, and the result looks good!
